# Clase ArduinoCommunication (Comunicación serie entre PC y Arduino)

El siguiente taller intenta explicar el funcionamiento de la clase *ArduinoCommunication* para comunicar la PC y el <span style="color:#F37263">**Arduino M1**</span>

Importamos librerias a utilizar

In [1]:
import serial
import time

### Iniciamos codificación para la clase ArduinoCommunication

Es importante siempre declarar un *constructor*. Esta función se llama cada vez que se genera un objeto. En nuestro caso, el constructor esta dado por,

```python
    def __init__(self, port, trialDuration = 6, stimONTime = 4,
                 timerFrecuency = 1000, timing = 1, useExternalTimer = False,
                 ntrials = 1)
```

Las variables del constructor se usan dentro del objeto para inicializar parámetros o atributos de la clase.

A modo de ejemplo, con los argumentos *stimONTime, trialDuration y timerFrecuency* seteamos una variable interna a la clase que nos permitirá llevar a cabo el control de cuanto tiempo estarán los estímulos encendidos. Esto lo hacemos de la siguiente manera,

```python
self.stimOFFTime = int((trialDuration - stimONTime))/timing*timerFrecuency
```

La variable *port* es un *string* y lo usaremos para decirle en qué puerto estará conectado nuestro Arduino.

La conexión con el puerte serie la iniciamos haciendo,
```python
self.dev = serial.Serial(port, baudrate=19200)
```

In [2]:
class ArduinoCommunication:
    """Clase para comunicación entre Arduino y PC utilizando la libreria PYSerial.
        Constructor del objeto ArduinoCommunication
        
        Parametros
        ----------
        port: String
            Puerto serie por el cual nos conectaremos
        trialDuration: int
            Duración total de un trial [en segundos]
        stimONTime: int
            Duración total en que los estímulos están encendidos
        timerFrecuency: int
            Variable para "simular" la frecuencia [en Hz] de interrupción del timer
        timing: int
            Variable para temporizar interrupción - Por defecto es 1[ms]
        useExternalTimer: bool
            En el caso de querer que el timer funcione con una interrupción externa
        ntrials: int
            Cantidad de trials a ejecutar. Una vez pasados los ntrials, se deja de transmitir y recibir
            información hacia y desde el Arduino - Por defecto el valor es 1[trial] - Si se quisiera una
            ejecución por tiempo indeterminado se debe hacer ntrials = None
            
        Retorna
        -------
        Nada        
    """
    def __init__(self, port, trialDuration = 6, stimONTime = 4,
                 timerFrecuency = 1000, timing = 1, useExternalTimer = False,
                 ntrials = 1):
        
        self.dev = serial.Serial(port, baudrate=19200)
        
        self.trialDuration =   int((trialDuration*timerFrecuency)/timing) #segundos
        self.stimONTime = int((stimONTime*timerFrecuency)/timing) #segundos
        self.stimOFFTime = int((trialDuration - stimONTime))/timing*timerFrecuency
        self.stimStatus = "on"
        self.trial = 1
        self.trialsNumber = ntrials
        
        self.sessionStatus = b"1" #sesión en marcha
        self.stimuliStatus = b"0" #los estimulos empiezan apagados
        self.moveOrder = b"0" #EL robot empieza en STOP
        """
        self.moveOrder
        - 0: STOP
        - 1: ADELANTE
        - 2: DERECHA
        - 3: ATRAS
        - 4: IZQUIERDA
        """
        self.systemControl = [self.sessionStatus,
                             self.stimuliStatus,
                             self.moveOrder]
         
        self.useExternalTimer = useExternalTimer
        self.timerEnable = 0
        self.timing = timing #en milisegundos
        self.timerFrecuency = timerFrecuency #1000Hz
        self.initialTime = 0 #tiempo inicial en milisegundos
        self.counter = 0
        self.timerInteFlag = 0 #flag para la interrupción del timer. DEBE ponerse a 0 para inicar una cuenta nueva.
        
        time.sleep(2) #esperamos 2 segundos para una correcta conexión

#### Definiendo métodos

Toda clase posee métodos o funciones que nos dejan interactuar con el objeto creado a partir de ésta. A continuación declaro los métodos de la clase.

In [3]:
    def timer(self):
        """Función para emular un timer como el de un microcontrolador"""
        if(self.timerInteFlag == 0 and
           time.time()*self.timerFrecuency - self.initialTime >= self.timing):
            self.initialTime = time.time()*self.timerFrecuency
            self.timerInteFlag = 1
            
    def iniTimer(self):
        """Iniciamos conteo del timer"""
        self.initialTime = time.time()*1000
        self.timerInteFlag = 0

Los métodos *timer()* e *iniTimer()* son dos métodos que le permiten a la clase ArduinoCommunication generar una temporización de los eventos internos a la misma. La idea es generar una interrupción cada cierto tiempo, similar a lo que se hace dentro de un microcontrolador.

Si se analiza el método *timer()* puede verse que su implementación es sencilla, aunque podría optimizarse. 

##### Enviando un byte: Método *query()*

Este método nos permite enviar un *byte* por el puerto serie de la PC al Arduino.

Cada vez que enviamos un byte -dado por el argumeno *message*- vamos a recibir una cadena de caracteres desde Arduino como respuesta.

In [4]:
    def query(self, byte):
        """Enviamos un byte a arduinot y recibimos un byte desde arduino
        
        Parametros
        ----------
        message (byte):
            Byte que se desa enviar por puerto serie.
        """
        self.dev.write(byte)#.encode('ascii')) #enviamos byte por el puerto serie
        respuesta = self.dev.readline().decode('ascii').strip() #recibimos una respuesta desde Arduino
        
        return respuesta

##### Enviando un mensaje a través de varios Bytes: Método sendMessage()

El método *query()* nos permite enviar **un byte**. Sin embargo, necesitamos enviar varios Bytes. Para esto usaremos el método *sendMessage()* el cual nos permite enviar una cadena de Bytes haciendo uso del método *query()*.

Este mensaje contiene información relevante para el Arduino, por ejemeplo, el estado de los estímulos, el comando a enviar al robot, entre otros.

In [5]:
    def sendMessage(self, message):
        """Función para enviar una lista de bytes con diferentes variables de estado
        hacia Arduino. Estas variables sirven para control de flujo del programa del
        Arduino.
        """
        incomingData = []
        for byte in message:
            incomingData.append(self.query(byte))
            
        return incomingData[-1] #Retorno los últimos bytes recibidos

##### Cerrando comunicación serie

Mediante el método *close()* cerramos la comunicación serie y liberamos el puerto COM.

In [6]:
    def close(self):
        """Cerramos comunicción serie"""
        self.dev.close()

##### Iniciando y finializando sesión

Es de suma importancia mantener un orden correcto en el flujo de trabajo y conocer el estado interno del sistema. Con la idea de lograr esto, se crearon dos métodos que nos permiten iniciar y finalizar una sesión de trabajo.

El método *iniSesion()* lo que hace es inicializar todas las variables asociadas al control de estímulos y movimiento del robot a sus estados iniciales. Así por ejemplo se hace,

```python
   self.moveOrder = b"0" #EL robot empieza en STOP
   ```
Lo anterior será enviado al <span style="color:#F37263">**Arduino M1**</span> y desde ahí al <span style="color:#008a3e">**Arduino M3**</span>, indicandole al robot el comando *STOP*.

In [7]:
    def iniSesion(self):
        """Se inicia sesión."""
        
        self.sessionStatus = b"1" #sesión en marcha
        self.stimuliStatus = b"1" #encendemos estímulos
        self.moveOrder = b"0" #EL robot empieza en STOP
        
        self.systemControl = [self.sessionStatus,
                             self.stimuliStatus,
                             self.moveOrder]
        
        estadoRobot = self.sendMessage(self.systemControl)
        print("Estado inicial del ROBOT:", estadoRobot)
        
        self.iniTimer()
        print("Sesión iniciada")
        print("Trial inicial")

        
    def endSesion(self):
        """Se finaliza sesión.
            Se envía información a Arduino para finalizar sesión. Se deben tomar acciones en el Arduino
            una vez recibido el mensaje.
        """
        
        self.sessionStatus = b"0" #sesión finalizada
        self.stimuliStatus = b"0" #finalizo estimulación
        self.moveOrder = b"0" #Paramos el rebot enviando un STOP
        self.systemControl = [self.sessionStatus,
                             self.stimuliStatus,
                             self.moveOrder]
        
        estadoRobot = self.sendMessage(self.systemControl)
        print("Estado final del ROBOT:", estadoRobot)
        print("Sesión Finalizada")
        print(f"Trial final {self.trial - 1}")

##### Control de trials

Recordar que un **Trial** es el tiempo en que *estamos estimulando* más el tiempo que *estamos sin estimular*. Controlaremos todo esto desde Python mediante el método <span style="color:blue">*trialControl()*</span>.

Su implementación es sencilla. La variable *self.counter* se incrementa en *1* con cada interrupción generada por el método *timer()*. Cuando *self.counter == self.stimONTime* implica que hemos alcanzado el tiempo que tenemos que estar estimulando y por lo tanto, enviamos un mensaje al <span style="color:#F37263">**Arduino M1**</span> para que apague los estíumulos. Esto lo hacemos así,

```python
if self.counter == self.stimONTime: #mandamos nuevo mensaje cuando comienza un trial

    self.systemControl[1] = b"0" #apagamos estímulos
    estadoRobot = self.sendMessage(self.systemControl)
   ```

De manera similiar cuando *self.counter == self.trialDuration* implica que llegamos al final del trial y debemos empezar uno nuevo. Enviamos un mensaje al <span style="color:#F37263">**Arduino M1**</span> para que encienda los estímulos. Esto lo hacemos así,

```python
if self.counter == self.trialDuration: 

    self.systemControl[1] = b"1"
    estadoRobot = self.sendMessage(self.systemControl)
    print(f"Fin trial {self.trial}")
    print("")
    self.trial += 1 #incrementamos un trial
    self.counter = 0 #reiniciamos timer
   ```

In [8]:
    def trialControl(self):
        """Función que ayuda a controlar los estímulos en arduino.
            - La variable self.counter es utilizada como contador para sincronizar
            los momentos en que los estímulos están encendidos o opagados.
            - Es IMPORTANTE para un correcto funcionamiento que la variable self.counter
            se incremente en 1 de un tiempo adecuado. Para esto se debe tener en cuenta
            las variables self.stimONTime y self.trialDuration
        """

        self.counter += 1
        
        if self.counter == self.stimONTime: #mandamos nuevo mensaje cuando comienza un trial
        
            self.systemControl[1] = b"0" #apagamos estímulos
            estadoRobot = self.sendMessage(self.systemControl)
             
        if self.counter == self.trialDuration: 
            
            self.systemControl[1] = b"1"
            estadoRobot = self.sendMessage(self.systemControl)
            print(f"Fin trial {self.trial}")
            print("")
            self.trial += 1 #incrementamos un trial
            self.counter = 0 #reiniciamos timer
            
        return self.trial

##### Control general

El método *generalControl()* nos sirve para tener un control del flujo de programa.

**IMPORTANTE:** En este ejemplo sólo se implementa el control de los trials. Pero en un futuro también estará a cargo de tomar los datos de EEG desde la placa OpenBCI y pasarsela a los módulos de procesamiento y clasificación que se encargarán de obtener un comando para el robot.

In [9]:
    def generalControl(self):
        """Función para llevar a cabo un control general de los procesos entre PC y Arduino."""
        
        if self.systemControl[0] == b"1" and not self.trialsNumber: #Para trials indefinidos
            
            if not self.useExternalTimer:
                self.timer()        
                
            if self.timerInteFlag: #timerInteFlag se pone en 1 a la cantidad de milisegundos de self.timing
                self.trialControl()
                self.timerInteFlag = 0 #reiniciamos flag de interrupción
                
        elif self.systemControl[0] == b"1" and self.trial <= self.trialsNumber: #Para cantidad de trials definido
            
            if not self.useExternalTimer:    
                self.timer()   
                
            if self.timerInteFlag: #timerInteFlag se pone en 1 a la cantidad de milisegundos de self.timing
                self.trialControl()
                self.timerInteFlag = 0 #reiniciamos flag de interrupción
        
        else:
            self.endSesion()
                
        return self.sessionStatus

### Testeando nuestro código

Ahora vamos a hacer una prueba de nuestro código.

Instanciamos un objeto ArduinoCommunication y le pasamos como parámetros el puerto *'COM3'*, duración del trial igual a *8 segundos*, duración de estimulación igual a *4 segundos* y número de trials igual a *2*. Esto lo hacemos así,

```python
    ard = ArduinoCommunication('COM3', trialDuration = 8, stimONTime = 4,
                               timing = 100, ntrials = 2)
```

In [10]:
def main():
    
    initialTime = time.time()#/1000

    """
    #creamos un objeto ArduinoCommunication para establecer una conexión
    #entre arduino y nuestra PC en el COM3, con un timing de 500ms y esperamos ejecutar
    #n trials.
    #Pasado estos n trials se finaliza la sesión.
    #En el caso de querer ejecutar Trials de manera indeterminada,
    #debe hacerse trials = None (default)
    """
    ard = ArduinoCommunication('COM3', trialDuration = 8, stimONTime = 4,
                               timing = 100, ntrials = 2)

    ard.iniSesion()
    
    while ard.generalControl() == b"1":
        pass

    #ard.endSesion()   
    ard.close() #cerramos comunicación serie y liberamos puerto COM
    
    stopTime = time.time()#/1000
    
    print(f"Tiempo transcurrido en segundos: {stopTime - initialTime}")

### IMPORTANTE

Para poder ejecutar el código, todo debe ser pasado a un archivo .py